In [1]:
from instagram_private_api import Client, ClientCompatPatch
import api_requests
import importlib
importlib.reload(api_requests)
from api_requests import BasicUserInfo
from api_requests import PublicUserInfoRequests
from api_requests import MediaInfoRequests

In [2]:
"""
Only run once, multiple log ins will cause account ban
TODO: Figure out how to cache auth cookie details. Hints: Can call api.cookie_jar, look into client.py
"""
username = 'cuteanimalzzzz123'
password = 'instabotOP'
api = Client(username, password)

In [3]:
test_usernames = ['ryanlopezzzz', #small size public account example
                  'ruining99', #small size public account example
                  'i_am_jorge_murillo', #private account example
                  'cuteanimalzzzz123' #API login account
                 ]
large_test_usernames = ['cute.animals'] #large size public account

# Test BasicUserInfo() Class

In [4]:
def test_basic_user_info(username):
    """
    Can check against true values by going onto Instagram app.
    """
    basic_user_info = BasicUserInfo(api, username)
    print("User Id: " + str(basic_user_info.user_id))
    print("Follow count: " + str(basic_user_info.follower_count))
    print("Following count: " + str(basic_user_info.following_count))
    print("Media count: " + str(basic_user_info.media_count))
    print("Private status: " + str(basic_user_info.private_status))

In [5]:
for username in test_usernames + large_test_usernames:
    print("Username: " + username)
    test_basic_user_info(username)
    print("-"*100)

Username: ryanlopezzzz
User Id: 255272463
Follow count: 636
Following count: 391
Media count: 14
Private status: False
----------------------------------------------------------------------------------------------------
Username: ruining99
User Id: 2142436680
Follow count: 387
Following count: 368
Media count: 15
Private status: False
----------------------------------------------------------------------------------------------------
Username: i_am_jorge_murillo
User Id: 9984230653
Follow count: 172
Following count: 190
Media count: 2
Private status: True
----------------------------------------------------------------------------------------------------
Username: cuteanimalzzzz123
User Id: 50004795460
Follow count: 0
Following count: 0
Media count: 0
Private status: False
----------------------------------------------------------------------------------------------------
Username: cute.animals
User Id: 435102802
Follow count: 88333
Following count: 1
Media count: 344
Private status: F

# Test PublicUserInfoRequests() Class

In [6]:
def test_public_user_info_requests(username):
    rank_token = api.generate_uuid()
    puir = PublicUserInfoRequests(api,username,rank_token)
    
    followers_usernames = puir.get_followers_usernames()
    assert(puir.follower_count == len(followers_usernames))
    print('Loaded %d follower usernames.'%len(followers_usernames))
    print('Peeking at first 3 followers: '+str(followers_usernames[:3]))
    print()    
    following_usernames = puir.get_following_usernames()
    assert(puir.following_count == len(following_usernames))
    print('Loaded %d following usernames.'%len(following_usernames))
    print('Peeking at first 3 following: '+str(following_usernames[:3]))
    print()
    user_feed_media_ids = puir.get_user_feed_media_ids()
    assert(puir.media_count == len(user_feed_media_ids))
    print('Loaded %d media (posts) ids.'%len(user_feed_media_ids))
    print('Peeking at first 2 ids: ' + str(user_feed_media_ids[:2]))

In [7]:
for username in test_usernames:
    print("Username: " + username + '\n')
    try:
        test_public_user_info_requests(username)
    except Exception as e:
        print('Exception: %s, Message: %s'%(type(e).__name__,e))
    print("-"*100)

Username: ryanlopezzzz

Loaded 636 follower usernames.
Peeking at first 3 followers: ['c.hirish', 'matthewkim812', 'r.yamas']

Loaded 391 following usernames.
Peeking at first 3 following: ['c.hirish', 'happy_lucky_now.jp', 'neha_upponi']

Loaded 14 media (posts) ids.
Peeking at first 2 ids: ['2112726015003571292_255272463', '2018080158682059371_255272463']
----------------------------------------------------------------------------------------------------
Username: ruining99

Loaded 387 follower usernames.
Peeking at first 3 followers: ['cassandra.chow', 'c.hirish', 'andrewarmen']

Loaded 368 following usernames.
Peeking at first 3 following: ['cassandra.chow', 'c.hirish', 'johannalimmer']

Loaded 15 media (posts) ids.
Peeking at first 2 ids: ['2601326525749798387_2142436680', '2580166803319609035_2142436680']
----------------------------------------------------------------------------------------------------
Username: i_am_jorge_murillo

Exception: ValueError, Message: Can't get foll

# Test MediaInfoRequests() Class
Seems there can be an error where it loads less comments than get_comment_count() says there are.

In [8]:
def test_media_info_requests(username):
    rank_token = api.generate_uuid()
    
    #Get user's most recent media id
    puir = PublicUserInfoRequests(api,username,rank_token)
    user_feed_media_ids = puir.get_user_feed_media_ids()
    most_recent_media_id = user_feed_media_ids[0]
    
    mir = MediaInfoRequests(api,most_recent_media_id,rank_token)
    print('Comment count: %d'%mir.get_comment_count())
    print()                       
    comments_text = mir.get_n_comments_text()
    print('Loaded %d comments text.'%len(comments_text))
    print('Peeking at first 2 comments: '+str(comments_text[:2]))
    print()
    comment_counter = mir.get_n_comment_count_per_username()
    assert(sum(comment_counter.values())==len(comments_text)) 
    print('Loaded %d (username, comment count) data.'%len(comment_counter))
    comment_counter_list = list(comment_counter.items())
    print('Peeking at first 2 elements: '+str(comment_counter_list[:2]))

In [ ]:
for username in test_usernames + large_test_usernames:
    print("Testing on %s's first post: \n"%username)
    try:
        test_media_info_requests(username)
    except Exception as e:
        print('Exception: %s, Message: %s'%(type(e).__name__,e))
    print("-"*100)

Testing on ryanlopezzzz's first post: 

Comment count: 7

Loaded 7 comments text.
Peeking at first 2 comments: ['this is so aesthetic i see u', 'Hahahaha i love the second photo so much']

Loaded 6 (username, comment count) data.
Peeking at first 2 elements: [('emmawilt', 1), ('nicoleqidess', 1)]
----------------------------------------------------------------------------------------------------
Testing on ruining99's first post: 

